# ObserveEvidence using pandas

In [3]:
%pylab inline
import itertools
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [4]:
tuples = list(itertools.product([1,2,3], [1,2], [1,2]))
mindex = pd.MultiIndex.from_tuples(tuples, names=['a', 'b', 'c'])
factor = pd.Series(
    [0.25, 0.35, 0.08, 0.16, 0.05, 0.07, 0.0, 0.0, 0.15, 0.21, 0.07, 0.14],
    index=mindex)
factor

a  b  c
1  1  1    0.25
      2    0.35
   2  1    0.08
      2    0.16
2  1  1    0.05
      2    0.07
   2  1    0.00
      2    0.00
3  1  1    0.15
      2    0.21
   2  1    0.07
      2    0.14
dtype: float64

**I'll encode evidence as a python dictionary: `{ variableName:value }`: **

In [5]:
evidence = { "b": 2 }

**The simplest way I've managed to come up with to Observe evidence that b=2.**  
**First I reset the evidence-variable from the index ... **

In [19]:
factor.reset_index(level="b")

b     0
a c         
1 1  1  0.25
  2  1  0.35
  1  2  0.08
  2  2  0.16
2 1  1  0.05
  2  1  0.07
  1  2  0.00
  2  2  0.00
3 1  1  0.15
  2  1  0.21
  1  2  0.07
  2  2  0.14

** ... then use boolean indexing to set the rest of the factor to zero**

In [30]:
resetV = factor.reset_index(level="b")
resetV.loc[resetV["b"] != 2, 0] = 0
resetV

b     0
a c         
1 1  1  0.00
  2  1  0.00
  1  2  0.08
  2  2  0.16
2 1  1  0.00
  2  1  0.00
  1  2  0.00
  2  2  0.00
3 1  1  0.00
  2  1  0.00
  1  2  0.07
  2  2  0.14

**Adding the observed variable to index and doing the loop over the dictionary:**

In [34]:
observed = factor.copy()
for variableName, value in evidence.iteritems():
    resetV = observed.reset_index(level=variableName)
    resetV.loc[resetV[variableName] != value, 0] = 0
    observed = resetV.set_index(variableName, append=True)[0]
observed

a  c  b
1  1  1    0.00
   2  1    0.00
   1  2    0.08
   2  2    0.16
2  1  1    0.00
   2  1    0.00
   1  2    0.00
   2  2    0.00
3  1  1    0.00
   2  1    0.00
   1  2    0.07
   2  2    0.14
Name: 0, dtype: float64

**Finally, lets make it a function:**

In [33]:
def ObserveEvidence(factor, **evidence):
    observed = factor.copy()
    for variableName, value in evidence.iteritems():
        resetV = observed.reset_index(level=variableName)
        resetV.loc[resetV[variableName] != value, 0] = 0
        observed = resetV.set_index(variableName, append=True)[0]
    return observed